In [1]:
# Import packages
import geopandas as gpd
import numpy as np
import pandas as pd
import libpysal
import networkx as nx
import osmnx as ox
import time
import os
from shapely import geometry
from shapely.geometry import Point, MultiLineString, LineString, Polygon
from shapely.ops import nearest_points
from itertools import product, combinations
import math
import warnings
# Import some libraries
import socket
from wpgpDownload.utils.dl import wpFtp
from wpgpDownload.utils.isos import Countries
from wpgpDownload.utils.convenience_functions import download_country_covariates as dl
from wpgpDownload.utils.wpcsv import Product
import georasters as gr

In [2]:
# Block 0 cities and assumptions

start = time.time()

cities = ['Philadelphia']

# Assumptions
thresholds = [300, 600, 1000] # route threshold in metres. WHO guideline speaks of access within 300m
walkable_park_dist = 500 # radius in metres
one_park_buffer = 25 # in metres
park_entry_point_buffer = 25 # in metres
min_park_size = 400 # in squared metres (WHO = 0.04 ha = 400m)

park_entry_point_merge = 0 # ! Be careful with siplyfying topology with this, can alter the scores quite a bit.
grid_block_size = 100 # in meters2 rounded to 100m
nn_iter = 25 # tries to find nearest nodes at route finding
#block_combinations = 100
block_network = 250000 # chunks for route finding

In [3]:
# Block 1 Extract city boundaries, download grids in cities' country

warnings.filterwarnings("ignore")
bound_df = ox.geocoder.geocode_to_gdf(cities)
bound_df.to_file(r'D:/Dumps/City_boundaries/Boundaries_osm.shp')

start_time = time.time()

# Get unique ISO countries, so all country-grids are only loaded once (i.e. Philadelphia, Denver and Washington DC in the USA)
C = pd.DataFrame(Countries)
start_time = time.time()
iso_countries = []
print('required WorldPoP files (execute code string(s) below by country in the Jupyter terminal)')
for i in range(len(cities)): 
    countries = bound_df['display_name'][i].rsplit(',')[-1][1:]
    iso = C[C['name'] == countries].iloc[0,1]
    if iso not in iso_countries:
        iso_countries.append(iso)
        
        # Search for data
        products = Product(iso)
        Results = products.description_contains('people per grid-cell 2020')
        list1 = []
        for p in Results:
            prints = '%s/%s\t%s\t%s' % (p.idx, p.country_name,p.dataset_name,p.path)
            list1.append(prints)
        print('wpgpDownload download -i',iso,'--id',list1[0].split("\t")[0].split('/')[0])

print('')
blocks = []
for i in iso_countries:
    # Get the country by saved path ()
    # You can also use sys.path: path = sys.path[0] +'\\' + str('ISL').lower() + '_ppp_2020.tif'
    # The sys.path is the path Python saves the python file automatically before you
    path = 'D:\\Dumps\\WorldPoP_Grids\\' + i.lower() + '_ppp_2020.tif'
    block = gr.from_file(path)
    blocks.append(block)
    print(i,'extracted', round((time.time() - start_time)/60,2),'mns')
bound_df

required WorldPoP files (execute code string(s) below by country in the Jupyter terminal)
wpgpDownload download -i USA --id 4983

USA extracted 1.2 mns


,geometry,bbox_north,bbox_south,bbox_east,bbox_west,place_id,osm_type,osm_id,lat,lon,display_name,class,type,importance
0,"POLYGON ((-75.28030 39.97500, -75.28022 39.974...",40.137959,39.867005,-74.955831,-75.280298,282310523,relation,188022,40.00241,-75.139364,"Philadelphia, Philadelphia County, Pennsylvani...",boundary,administrative,0.823797


In [4]:
# Block 2 population grids extraction
start_time = time.time()
clips = []
grids = []
print(str(grid_block_size) + 'm resolution grids extraction')
for i in range(len(cities)):
    iso = C[bound_df['display_name'][i].rsplit(',')[-1][1:] == C['name']].iloc[0,1]
    contains = [j for j, x in enumerate(iso_countries) if x == iso][0]
    
    # Clip the city from the country
    clipped = blocks[contains].clip(bound_df['geometry'][i])
    clipped = clipped[0].to_geopandas()
    
    # Get dissolvement_key for dissolvement. 
    clipped['row3'] = np.floor(clipped['row']/(grid_block_size/100)).astype(int)
    clipped['col3'] = np.floor(clipped['col']/(grid_block_size/100)).astype(int)
    clipped['dissolve_key'] = clipped['row3'].astype(str) +'-'+ clipped['col3'].astype(str)
    clips.append(clipped)
        
    # Dissolve into block by block grids
    popgrid = clipped[['dissolve_key','geometry','row3','col3']].dissolve('dissolve_key')

    # Get those grids populations and area. Only blocks with population and full blocks
    popgrid['population'] = round(clipped.groupby('dissolve_key')['value'].sum()).astype(int)
    popgrid['area_m'] = round(gpd.GeoSeries(popgrid['geometry'], crs = 4326).to_crs(3043).area).astype(int)
    popgrid = popgrid[popgrid['population'] > 0]
    popgrid = popgrid[popgrid['area_m'] / popgrid['area_m'].max() > 0.95]
    
    # Get centroids and coords
    popgrid['centroid'] = popgrid['geometry'].centroid
    popgrid['centroid_m'] = gpd.GeoSeries(popgrid['centroid'], crs = 4326).to_crs(3043)
    popgrid['grid_lon'] = popgrid['centroid_m'].x
    popgrid['grid_lat'] = popgrid['centroid_m'].y
    popgrid = popgrid.reset_index()
    
    minx = popgrid.bounds['minx']
    maxx = popgrid.bounds['maxx']
    miny = popgrid.bounds['miny']
    maxy = popgrid.bounds['maxy']

    # Some geometries result in a multipolygon when dissolving (like i.e. 0.05 meters) which is in my mind an coords error
    # I therefore create one polygon
    Poly = []
    for k in range(len(popgrid)):
        Poly.append(Polygon([(minx[k],maxy[k]),(maxx[k],maxy[k]),(maxx[k],miny[k]),(minx[k],miny[k])]))
    popgrid['geometry'] = Poly
    
    grids.append(popgrid)
    
    print(cities[i].rsplit(',')[0], round((time.time() - start_time)/60,2),'mns')
    
grids[0]

100m resolution grids extraction
Philadelphia 0.23 mns


,dissolve_key,geometry,row3,col3,population,area_m,centroid,centroid_m,grid_lon,grid_lat
0,0-318,"POLYGON ((-75.01542 40.13875, -75.01458 40.138...",0,318,10,14838,POINT (-75.01500 40.13833),POINT (-5675320.992 8459941.626),-5.675321e+06,8.459942e+06
1,1-317,"POLYGON ((-75.01625 40.13792, -75.01542 40.137...",1,317,4,14839,POINT (-75.01583 40.13750),POINT (-5675486.353 8459999.741),-5.675486e+06,8.460000e+06
2,1-318,"POLYGON ((-75.01542 40.13792, -75.01458 40.137...",1,318,12,14839,POINT (-75.01500 40.13750),POINT (-5675453.122 8459898.329),-5.675453e+06,8.459898e+06
3,1-319,"POLYGON ((-75.01458 40.13792, -75.01375 40.137...",1,319,9,14838,POINT (-75.01417 40.13750),POINT (-5675419.890 8459796.917),-5.675420e+06,8.459797e+06
4,1-320,"POLYGON ((-75.01375 40.13792, -75.01292 40.137...",1,320,6,14838,POINT (-75.01333 40.13750),POINT (-5675386.655 8459695.507),-5.675387e+06,8.459696e+06
...,...,...,...,...,...,...,...,...,...,...
34885,99-95,"POLYGON ((-75.20125 40.05625, -75.20042 40.056...",99,95,2,14926,POINT (-75.20083 40.05583),POINT (-5695801.873 8478348.566),-5.695802e+06,8.478349e+06
34886,99-96,"POLYGON ((-75.20042 40.05625, -75.19958 40.056...",99,96,4,14926,POINT (-75.20000 40.05583),POINT (-5695768.954 8478246.652),-5.695769e+06,8.478247e+06
34887,99-97,"POLYGON ((-75.19958 40.05625, -75.19875 40.056...",99,97,8,14926,POINT (-75.19917 40.05583),POINT (-5695736.034 8478144.739),-5.695736e+06,8.478145e+06
34888,99-98,"POLYGON ((-75.19875 40.05625, -75.19792 40.056...",99,98,8,14926,POINT (-75.19833 40.05583),POINT (-5695703.111 8478042.827),-5.695703e+06,8.478043e+06


In [5]:
# Block 3 Road networks

warnings.filterwarnings("ignore")

start_time = time.time()
graphs = list()
road_nodes = list()
road_edges = list()
road_conn = list()

for i in cities:
    # Get graph, road nodes and edges
    graph = ox.graph_from_place(i, network_type = "all", buffer_dist = (np.max(thresholds)+1000))
    #graphs.append(graph)
    
    road_node, road_edge = ox.graph_to_gdfs(graph)
    
    # Road nodes format
    road_node = road_node.to_crs(4326)
    road_node['geometry_m'] = gpd.GeoSeries(road_node['geometry'], crs = 4326).to_crs(3043)
    road_node['osmid_var'] = road_node.index
    road_node = gpd.GeoDataFrame(road_node, geometry = 'geometry', crs = 4326)
    #road_nodes.append(road_node)
    
    # format road edges
    road_edge = road_edge.to_crs(4326)
    road_edge['geometry_m'] = gpd.GeoSeries(road_edge['geometry'], crs = 4326).to_crs(3043)
    road_edge = road_edge.reset_index()
    road_edge.rename(columns={'u':'from', 'v':'to', 'key':'keys'}, inplace=True)
    road_edge['key'] = road_edge['from'].astype(str) + '-' + road_edge['to'].astype(str)
    #road_edges.append(road_edge)
    
    # Exclude highways and ramps on edges    
    edge_filter = road_edge[(road_edge['highway'].str.contains('motorway') | 
          (road_edge['highway'].str.contains('trunk') & 
           road_edge['maxspeed'].astype(str).str.contains(
               '40 mph|45 mph|50 mph|55 mph|60 mph|65|70|75|80|85|90|95|100|110|120|130|140'))) == False]
    road_edges.append(edge_filter)
    
    # Exclude isolated nodes
    fltrnodes = pd.Series(list(edge_filter['from']) + list(edge_filter['to'])).unique()
    newnodes = road_node[road_node['osmid_var'].isin(fltrnodes)]
    road_nodes.append(newnodes)
    
    # Get only necessary road connections columns for network performance
    road_con = edge_filter[['osmid','key','length','geometry']]
    road_con = road_con.set_index('key')
    road_conn.append(road_con)
    
    # formatting to graph again.
    newnodes = newnodes.loc[:, ~newnodes.columns.isin(['geometry_m', 'osmid_var'])]
    edge_filter = edge_filter.set_index(['from','to','keys'])
    edge_filter = edge_filter.loc[:, ~edge_filter.columns.isin(['geometry_m', 'key'])]
    graph2 = ox.graph_from_gdfs(newnodes, edge_filter)
                            
    graphs.append(graph2)
    print(i.rsplit(',')[0], 'done', round((time.time() - start_time) / 60,2),'mns')
    
road_edges[0]

Philadelphia done 2.39 mns


,from,to,keys,osmid,name,highway,oneway,length,geometry,ref,...,width,bridge,service,tunnel,maxspeed,access,junction,area,geometry_m,key
0,103237949,103353127,0,11591597,New Jersey Avenue,residential,False,111.799,"LINESTRING (-74.96444 40.03625, -74.96557 40.0...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-5689489.546 8448457.709, -5689617...",103237949-103353127
1,103237949,103353090,0,11591597,New Jersey Avenue,residential,False,107.489,"LINESTRING (-74.96444 40.03625, -74.96337 40.0...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-5689489.546 8448457.709, -5689364...",103237949-103353090
2,103237949,103237976,0,11580386,Cleveland Avenue,residential,False,168.717,"LINESTRING (-74.96444 40.03625, -74.96404 40.0...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-5689489.546 8448457.709, -5689552...",103237949-103237976
3,103237976,103238007,0,11580386,Cleveland Avenue,residential,False,151.349,"LINESTRING (-74.96340 40.03496, -74.96245 40.0...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-5689653.040 8448263.511, -5689797...",103237976-103238007
4,103237976,103590312,0,11610261,2nd Street,residential,False,108.508,"LINESTRING (-74.96340 40.03496, -74.96448 40.0...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-5689653.040 8448263.511, -5689779...",103237976-103590312
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
338097,9726316545,5550941352,0,579660432,NaN,service,False,39.965,"LINESTRING (-75.24128 40.06507, -75.24111 40.0...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-5695926.465 8483769.053, -5695935...",9726316545-5550941352
338098,9726316545,9726316548,0,1058487388,NaN,service,False,58.733,"LINESTRING (-75.24128 40.06507, -75.24148 40.0...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-5695926.465 8483769.053, -5695962...",9726316545-9726316548
338099,9726316548,9726316545,0,1058487388,NaN,service,False,58.733,"LINESTRING (-75.24175 40.06470, -75.24175 40.0...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-5696003.982 8483808.258, -5695998...",9726316548-9726316545
338100,9728655691,110122187,0,12149073,South 10th Street,residential,True,3.250,"LINESTRING (-75.15829 39.94437, -75.15830 39.9...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-5711864.888 8467388.116, -5711869...",9728655691-110122187


In [6]:
# Block 4 city greenspace
parks_in_range = list()
for i in cities:
    gdf = ox.geometries_from_place(i, tags={'leisure':'park'}, buffer_dist = np.max(thresholds))
    gdf = gdf[(gdf.geom_type == 'Polygon') | (gdf.geom_type == 'MultiPolygon')]
    greenspace = gdf.reset_index()    
    warnings.filterwarnings("ignore")
    
    green_buffer = gpd.GeoDataFrame(geometry = greenspace.to_crs(3043).buffer(one_park_buffer).to_crs(4326))
    greenspace['geometry_w_buffer'] = green_buffer
    greenspace['geometry_w_buffer'] = gpd.GeoSeries(greenspace['geometry_w_buffer'], crs = 4326)
    greenspace['geom buffer diff'] = greenspace['geometry_w_buffer'].difference(greenspace['geometry'])

    # This function group components in itself that overlap (with the buffer set of 25 metres)
    # https://stackoverflow.com/questions/68036051/geopandas-self-intersection-grouping
    W = libpysal.weights.fuzzy_contiguity(greenspace['geometry_w_buffer'])
    greenspace['components'] = W.component_labels
    parks = greenspace.dissolve('components')
    
    # Exclude parks below 0.04 ha.
    parks = parks[parks.to_crs(3043).area > min_park_size]
    print(i, 'done')
    parks = parks.reset_index()
    parks_in_range.append(parks)
parks_in_range[0]

Philadelphia done


,components,geometry,element_type,osmid,ele,gnis:county_id,gnis:created,gnis:feature_id,gnis:state_id,leisure,...,dog,wheelchair,name:fr,playground:theme,lit,ways,type,protect_class,geometry_w_buffer,geom buffer diff
0,0,"MULTIPOLYGON (((-75.14967 39.94550, -75.14974 ...",way,25271691,None,None,None,None,None,park,...,None,no,Parc national historique de l'indépendance,None,None,"[805522112, 805522108, 805522110, 805522113, 8...",boundary,None,"POLYGON ((-75.15047 39.95057, -75.15047 39.950...","POLYGON ((-75.15047 39.95058, -75.15046 39.950..."
1,1,"MULTIPOLYGON (((-75.15048 39.93406, -75.15033 ...",way,25430119,None,None,None,1178025,None,park,...,None,None,None,None,None,None,None,None,"POLYGON ((-75.15094 39.93413, -75.15093 39.934...","POLYGON ((-75.15093 39.93414, -75.15093 39.934..."
2,2,"POLYGON ((-75.15210 39.92832, -75.15082 39.928...",way,25430697,None,None,None,1173253,None,park,...,None,None,None,None,None,None,None,None,"POLYGON ((-75.15229 39.92835, -75.15229 39.928...","POLYGON ((-75.15229 39.92836, -75.15228 39.928..."
3,3,"POLYGON ((-75.14239 39.96532, -75.14212 39.966...",way,28311898,None,None,None,None,None,park,...,None,None,None,None,None,None,None,None,"POLYGON ((-75.14241 39.96517, -75.14243 39.965...","POLYGON ((-75.14243 39.96517, -75.14245 39.965..."
4,4,"POLYGON ((-75.16962 39.96199, -75.16822 39.961...",way,28533992,None,None,None,None,None,park,...,None,None,None,None,None,None,None,None,"POLYGON ((-75.16981 39.96201, -75.16981 39.962...","POLYGON ((-75.16981 39.96203, -75.16980 39.962..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
444,460,"POLYGON ((-75.16529 40.00657, -75.16565 40.006...",way,1044598682,None,None,None,None,None,park,...,None,None,None,None,None,None,None,None,"POLYGON ((-75.16510 40.00655, -75.16513 40.006...","POLYGON ((-75.16513 40.00642, -75.16513 40.006..."
445,461,"MULTIPOLYGON (((-75.20883 39.94693, -75.20860 ...",relation,2588200,None,None,None,None,None,park,...,None,None,None,None,None,"[30863033, 30856590]",multipolygon,None,"POLYGON ((-75.21060 39.94844, -75.21131 39.949...","POLYGON ((-75.21131 39.94905, -75.21133 39.949..."
446,462,"POLYGON ((-75.16398 39.93837, -75.16413 39.938...",relation,3924908,None,None,None,None,None,park,...,None,None,None,None,None,"[494236932, 494236930, 494236933]",multipolygon,None,"POLYGON ((-75.16419 39.93825, -75.16420 39.938...","POLYGON ((-75.16420 39.93825, -75.16422 39.938..."
447,463,"MULTIPOLYGON (((-75.09761 39.98238, -75.09760 ...",relation,12989584,None,None,None,None,None,park,...,None,None,None,None,None,"[521268458, 744197596]",multipolygon,None,"MULTIPOLYGON (((-75.09845 39.98360, -75.09844 ...","MULTIPOLYGON (((-75.09844 39.98359, -75.09843 ..."


In [7]:
# Block 5 park entry points
start_time = time.time()
ParkRoads = list()
for j in range(len(cities)):
    ParkRoad = pd.DataFrame()
    mat = list()
    # For all
    for i in range(len(parks_in_range[j])):
        dist = road_nodes[j]['geometry'].to_crs(3043).distance(parks_in_range[j]['geometry'].to_crs(
            3043)[i])
        buf_nodes = road_nodes[j][(dist < park_entry_point_buffer) & (dist > 0)]
        mat.append(list(np.repeat(i, len(buf_nodes))))
        ParkRoad = pd.concat([ParkRoad, buf_nodes])
        if i % 50 == 0: print(cities[j].rsplit(',')[0], round(i/len(parks_in_range[j])*100,1),'% done', 
                              round((time.time() - start_time) / 60,2),' mns')
    # Park no list conversion
    mat_u = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, mat) for i in b]

    # Format
    ParkRoad['Park_No'] = mat_u
    ParkRoad = ParkRoad.reset_index()
    ParkRoad['park_lon'] = ParkRoad['geometry_m'].x
    ParkRoad['park_lat'] = ParkRoad['geometry_m'].y
    
    # Get the road nodes intersecting with the parks' buffer
    ParkRoad = pd.merge(ParkRoad, parks_in_range[j][['geometry']], left_on = 'Park_No', right_index = True)
    
    # Get the walkable park size
    ParkRoad['park_size_walkable'] = ParkRoad['geometry_m'].buffer(walkable_park_dist).to_crs(4326).intersection(ParkRoad['geometry_y'])
    ParkRoad['walk_area'] = ParkRoad['park_size_walkable'].to_crs(3043).area
    ParkRoad['park_area'] = ParkRoad['geometry_y'].to_crs(3043).area
    ParkRoad['share_walked'] = ParkRoad['walk_area'] / ParkRoad['park_area']
    
    # Get size inflation factors for the gravity model
    ParkRoad['size_infl_factor'] = ParkRoad['walk_area'] / ParkRoad['walk_area'].median()
    ParkRoad['size_infl_sqr2'] = ParkRoad['size_infl_factor']**(1/2)
    ParkRoad['size_infl_sqr3'] = ParkRoad['size_infl_factor']**(1/3)
    ParkRoad['size_infl_sqr5'] = ParkRoad['size_infl_factor']**(1/5)
    ParkRoads.append(ParkRoad)
    
    fltr = ParkRoad.loc[:,~ParkRoad.columns.isin(['geometry_x','geometry_m','park_size_walkable', 
                                                          'geometry_m_buffer'])]
                     
    gdf = gpd.GeoDataFrame(pd.DataFrame(fltr), geometry = 'geometry_y', crs = 4326)
    gdf.to_file('D:Dumps/Scores output OSM/Park_entry_points/'+ cities[j] +'.shp')
    
    print(cities[j].rsplit(',')[0],'100 % done', 
                              round((time.time() - start_time) / 60,2),' mns')
    
ParkRoads[0]

Philadelphia 0.0 % done 0.02  mns
Philadelphia 11.1 % done 0.4  mns
Philadelphia 22.3 % done 0.66  mns
Philadelphia 33.4 % done 0.95  mns
Philadelphia 44.5 % done 1.25  mns
Philadelphia 55.7 % done 1.51  mns
Philadelphia 66.8 % done 1.77  mns
Philadelphia 78.0 % done 2.03  mns
Philadelphia 89.1 % done 2.28  mns
Philadelphia 100 % done 2.68  mns


,osmid,y,x,street_count,highway,ref,geometry_x,geometry_m,osmid_var,Park_No,...,park_lat,geometry_y,park_size_walkable,walk_area,park_area,share_walked,size_infl_factor,size_infl_sqr2,size_infl_sqr3,size_infl_sqr5
0,109729372,39.956835,-75.147299,4,NaN,NaN,POINT (-75.14730 39.95683),POINT (-5709440.723 8466685.277),109729372,0,...,8.466685e+06,"MULTIPOLYGON (((-75.14967 39.94550, -75.14974 ...","MULTIPOLYGON (((-75.14345 39.95648, -75.14348 ...",129260.205233,766205.364911,0.168702,1.724732,1.313291,1.199243,1.115178
1,109729474,39.952591,-75.148474,4,traffic_signals,NaN,POINT (-75.14847 39.95259),POINT (-5710163.665 8466609.499),109729474,0,...,8.466609e+06,"MULTIPOLYGON (((-75.14967 39.94550, -75.14974 ...","MULTIPOLYGON (((-75.14638 39.95008, -75.14671 ...",152813.334108,766205.364911,0.199442,2.039004,1.427937,1.268059,1.153144
2,109729486,39.953330,-75.148318,5,NaN,NaN,POINT (-75.14832 39.95333),POINT (-5710039.650 8466628.565),109729486,0,...,8.466629e+06,"MULTIPOLYGON (((-75.14967 39.94550, -75.14974 ...","MULTIPOLYGON (((-75.14539 39.95528, -75.14543 ...",173200.754874,766205.364911,0.226050,2.311035,1.520209,1.322114,1.182392
3,109729661,39.956254,-75.147625,4,NaN,NaN,POINT (-75.14763 39.95625),POINT (-5709546.213 8466695.127),109729661,0,...,8.466695e+06,"MULTIPOLYGON (((-75.14967 39.94550, -75.14974 ...","MULTIPOLYGON (((-75.14376 39.95649, -75.14375 ...",160141.739542,766205.364911,0.209006,2.136787,1.461775,1.288014,1.163998
4,109729673,39.956857,-75.147486,4,NaN,NaN,POINT (-75.14749 39.95686),POINT (-5709444.553 8466709.353),109729673,0,...,8.466709e+06,"MULTIPOLYGON (((-75.14967 39.94550, -75.14974 ...","MULTIPOLYGON (((-75.14363 39.95651, -75.14367 ...",135357.990228,766205.364911,0.176660,1.806095,1.343910,1.217812,1.125507
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5642,7098423781,39.966995,-75.128847,1,NaN,NaN,POINT (-75.12885 39.96700),POINT (-5707086.662 8464950.373),7098423781,448,...,8.464950e+06,"MULTIPOLYGON (((-75.12728 39.96599, -75.12739 ...","MULTIPOLYGON (((-75.12739 39.96589, -75.12747 ...",73431.800154,73431.800154,1.000000,0.979808,0.989852,0.993223,0.995929
5643,7098423782,39.966951,-75.128758,4,crossing,NaN,POINT (-75.12876 39.96695),POINT (-5707090.110 8464937.109),7098423782,448,...,8.464937e+06,"MULTIPOLYGON (((-75.12728 39.96599, -75.12739 ...","MULTIPOLYGON (((-75.12739 39.96589, -75.12747 ...",73431.800154,73431.800154,1.000000,0.979808,0.989852,0.993223,0.995929
5644,8151346056,39.967180,-75.128285,3,NaN,NaN,POINT (-75.12829 39.96718),POINT (-5707034.735 8464891.123),8151346056,448,...,8.464891e+06,"MULTIPOLYGON (((-75.12728 39.96599, -75.12739 ...","MULTIPOLYGON (((-75.12739 39.96589, -75.12747 ...",73431.800154,73431.800154,1.000000,0.979808,0.989852,0.993223,0.995929
5645,8912442744,39.967246,-75.128846,4,NaN,NaN,POINT (-75.12885 39.96725),POINT (-5707046.718 8464963.244),8912442744,448,...,8.464963e+06,"MULTIPOLYGON (((-75.12728 39.96599, -75.12739 ...","MULTIPOLYGON (((-75.12739 39.96589, -75.12747 ...",73431.800154,73431.800154,1.000000,0.979808,0.989852,0.993223,0.995929


In [8]:
#ParkRoads[0]['geometry_x'].to_file('D:/Dumps/Chandigarh_nodes.shp')

In [9]:
# Block 5.5 (not in use, buffer is 0, thus retains all the park entry points as is)

# Get buffer of nodes close to each other.
ParkCombs = list([])
for i in range(len(cities)):
    
    # Get the buffer
    ParkComb = ParkRoads[i]
    ParkComb['geometry_m_buffer'] = ParkComb['geometry_m'].buffer(park_entry_point_merge)
    
    # Get and merge components
    M = libpysal.weights.fuzzy_contiguity(ParkComb['geometry_m_buffer'])
    ParkComb['components'] = M.component_labels
    
    # Take centroid of merged components
    centr = gpd.GeoDataFrame(ParkComb, geometry = 'geometry_x', crs = 4326).dissolve('components')['geometry_x'].centroid
    centr = gpd.GeoDataFrame(centr)
    centr.columns = ['comp_centroid']
    
    # Get node closest to the centroid of all merged nodes, which accesses the road network.
    ParkComb = pd.merge(ParkComb, centr, left_on = 'components', right_index = True)
    ParkComb['centr_dist'] = ParkComb['geometry_x'].distance(ParkComb['comp_centroid'])
    ParkComb = ParkComb.iloc[ParkComb.groupby('components')['centr_dist'].idxmin()]
    ParkCombs.append(ParkComb)
ParkCombs[0]

,osmid,y,x,street_count,highway,ref,geometry_x,geometry_m,osmid_var,Park_No,...,park_area,share_walked,size_infl_factor,size_infl_sqr2,size_infl_sqr3,size_infl_sqr5,geometry_m_buffer,components,comp_centroid,centr_dist
0,109729372,39.956835,-75.147299,4,NaN,NaN,POINT (-75.14730 39.95683),POINT (-5709440.723 8466685.277),109729372,0,...,766205.364911,0.168702,1.724732,1.313291,1.199243,1.115178,POLYGON EMPTY,0,POINT (-75.14730 39.95683),0.0
1,109729474,39.952591,-75.148474,4,traffic_signals,NaN,POINT (-75.14847 39.95259),POINT (-5710163.665 8466609.499),109729474,0,...,766205.364911,0.199442,2.039004,1.427937,1.268059,1.153144,POLYGON EMPTY,1,POINT (-75.14847 39.95259),0.0
2,109729486,39.953330,-75.148318,5,NaN,NaN,POINT (-75.14832 39.95333),POINT (-5710039.650 8466628.565),109729486,0,...,766205.364911,0.226050,2.311035,1.520209,1.322114,1.182392,POLYGON EMPTY,2,POINT (-75.14832 39.95333),0.0
3,109729661,39.956254,-75.147625,4,NaN,NaN,POINT (-75.14763 39.95625),POINT (-5709546.213 8466695.127),109729661,0,...,766205.364911,0.209006,2.136787,1.461775,1.288014,1.163998,POLYGON EMPTY,3,POINT (-75.14763 39.95625),0.0
4,109729673,39.956857,-75.147486,4,NaN,NaN,POINT (-75.14749 39.95686),POINT (-5709444.553 8466709.353),109729673,0,...,766205.364911,0.176660,1.806095,1.343910,1.217812,1.125507,POLYGON EMPTY,4,POINT (-75.14749 39.95686),0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5642,7098423781,39.966995,-75.128847,1,NaN,NaN,POINT (-75.12885 39.96700),POINT (-5707086.662 8464950.373),7098423781,448,...,73431.800154,1.000000,0.979808,0.989852,0.993223,0.995929,POLYGON EMPTY,5642,POINT (-75.12885 39.96700),0.0
5643,7098423782,39.966951,-75.128758,4,crossing,NaN,POINT (-75.12876 39.96695),POINT (-5707090.110 8464937.109),7098423782,448,...,73431.800154,1.000000,0.979808,0.989852,0.993223,0.995929,POLYGON EMPTY,5643,POINT (-75.12876 39.96695),0.0
5644,8151346056,39.967180,-75.128285,3,NaN,NaN,POINT (-75.12829 39.96718),POINT (-5707034.735 8464891.123),8151346056,448,...,73431.800154,1.000000,0.979808,0.989852,0.993223,0.995929,POLYGON EMPTY,5644,POINT (-75.12829 39.96718),0.0
5645,8912442744,39.967246,-75.128846,4,NaN,NaN,POINT (-75.12885 39.96725),POINT (-5707046.718 8464963.244),8912442744,448,...,73431.800154,1.000000,0.979808,0.989852,0.993223,0.995929,POLYGON EMPTY,5645,POINT (-75.12885 39.96725),0.0


In [12]:
# Block 6 grid-parkentry combinations within euclidean threshold distance

start_time = time.time()
RoadComb = list()
for l in range(len(cities)):
    #blockA = block_combinations
    print(cities[l])
    len1 = len(grids[l])
    len2 = len(ParkCombs[l])
    
    # Reduce the size of combinations per iteration
    len4 = 1
    len5 = len1 * len2
    blockC = len5
    while blockC > 1000000:
        blockC = len5 / len4
        #print(blockC, A4)
        len4 = len4+1
        
    # Amount of grids taken per iteration block
    block = int(np.ceil(len1 / len4))
    
    # Check number of blocks
    len3 = int(np.ceil(len2/block))
    
    output = pd.DataFrame()
    len_mat = 0
    # Checking all the combinations at once is too performance intensive, it is broken down per 1000 (or what you want)
    for i in range(len3):
        # Check all grid-park combinations per block
        l1, l2 = range(0,len1), range(i*block,(i+1)*block)
        listed = pd.DataFrame(list(product(l1, l2)))

        # Merge grid and park information
        grid_merged = pd.merge(listed, 
                               grids[l][['grid_lon','grid_lat','centroid','centroid_m']],
                               left_on = 0, right_index = True)
        node_merged = pd.merge(grid_merged, 
                               ParkCombs[l][['Park_No','osmid','geometry_x','geometry_y','geometry_m','park_lon','park_lat',
                                   'size_infl_sqr2','size_infl_sqr3','size_infl_sqr5','share_walked','park_area','walk_area']], 
                               left_on = 1, right_index = True)

        # Preset index for merging
        node_merged['key'] = range(0,len(node_merged))
        node_merged = node_merged.set_index('key')
        node_merged = node_merged.loc[:, ~node_merged.columns.isin(['index'])]

        # Create lists for better computational performance
        glon = list(node_merged['grid_lon'])
        glat = list(node_merged['grid_lat'])
        plon = list(node_merged['park_lon'])
        plat = list(node_merged['park_lat'])
        infl2 = list(node_merged['size_infl_sqr2'])
        infl3 = list(node_merged['size_infl_sqr3'])
        infl5 = list(node_merged['size_infl_sqr5'])

        # Get the euclidean distances
        mat = list()
        mat2 = list()
        mat3 = list()
        mat4 = list()
        for j in range(len(node_merged)):
            mat.append(math.sqrt(abs(plon[j] - glon[j])**2 + abs(plat[j] - glat[j])**2))
            mat2.append(math.sqrt(abs(plon[j] - glon[j])**2 + abs(plat[j] - glat[j])**2) / infl2[j])
            mat3.append(math.sqrt(abs(plon[j] - glon[j])**2 + abs(plat[j] - glat[j])**2) / infl3[j])
            mat4.append(math.sqrt(abs(plon[j] - glon[j])**2 + abs(plat[j] - glat[j])**2) / infl5[j])

        # Check if distances are within 1000m and join remaining info and concat in master df per 1000.
        mat_df = pd.DataFrame(mat3)[(np.array(mat) <= np.max(thresholds)) | 
                                    (np.array(mat2) <= np.max(thresholds)) | 
                                    (np.array(mat3) <= np.max(thresholds)) | 
                                    (np.array(mat4) <= np.max(thresholds))]

        # join the other gravity euclidean scores and other information
        mat_df = mat_df.join(pd.DataFrame(mat), lsuffix='_infl', rsuffix='_entr', how = 'left')
        mat_df = mat_df.join(pd.DataFrame(mat2), lsuffix='_entry', rsuffix='_pwr', how = 'left')
        mat_df = mat_df.join(pd.DataFrame(mat4), lsuffix='_pwr', rsuffix='_root', how = 'left')
        mat_df.columns = ['size_infl_eucl2','raw euclidean','size_infl_eucl3','size_infl_eucl5']    
        mat_df = mat_df.join(node_merged)

        output = pd.concat([output, mat_df])
        len_mat = len_mat + len(mat_df)
        
    
        if ((i+1) % 3 | ((i+1) == len3): 
            print((i+1),'/',len3,'comb. done',round((time.time() - start_time) / 60,2),' mns')
        if ((i+1) % 3 | ((i+1) == len3): 
            print('of',np.where(i+1 == len3, len2 % block * block, len1*block) ,
                                'within a Gravity model variant in one of',thresholds,'m threshold:',len_mat)
            len_mat = 0
    print((i+1) ,'/', len3)
    # Renaming columns
    print('total combinations within distance',len_mat)
    
    output.columns = ['size_infl_eucl3','raw euclidean','size_infl_eucl2','size_infl_eucl5',
                      'Grid_No','Park_entry_No','grid_lon','grid_lat','Grid_coords_centroid','Grid_m_centroid',
                      'Park_No','Parkroad_osmid','Park_geom','Parkroad_coords_centroid','Parkroad_m_centroid',
                      'park_lon','park_lat','size_infl_sqr2','size_infl_sqr3','size_infl_sqr5',
                      'parkshare_walked','park_area','walk_area_m2']
    
    output = output[['raw euclidean','size_infl_eucl2','size_infl_eucl3','size_infl_eucl5',
                     'Grid_No','Park_entry_No','Grid_coords_centroid','Grid_m_centroid',
                      'Park_No','Parkroad_osmid','Park_geom','Parkroad_coords_centroid','Parkroad_m_centroid',
                     'walk_area_m2','size_infl_sqr2','size_infl_sqr3','size_infl_sqr5']]
    
    # Reinstate geographic elements
    output = gpd.GeoDataFrame(output, geometry = 'Grid_coords_centroid', crs = 4326)
    output['Grid_m_centroid'] = gpd.GeoSeries(output['Grid_m_centroid'], crs = 3043)
    output['Parkroad_coords_centroid'] = gpd.GeoSeries(output['Parkroad_coords_centroid'], crs = 4326)
    output['Parkroad_m_centroid'] = gpd.GeoSeries(output['Parkroad_m_centroid'], crs = 3043)
    
    # Get the nearest entrance point for the grid centroids
    mat5 = list()
    for i in range(len(output)):
        try:
            nearest = int(road_nodes[l]['geometry'].sindex.nearest(output['Grid_coords_centroid'].iloc[i])[1])
            mat5.append(road_nodes[l]['osmid_var'].iloc[nearest])
        except:
            nearest = int(road_nodes[l]['geometry'].sindex.nearest(output['Grid_coords_centroid'].iloc[i])[1][0])
            mat5.append(road_nodes[l]['osmid_var'].iloc[nearest])
        if i % 250000 == 0: print(round(i/len(output)*100,1),'% gridentry done', round((time.time() - start_time) / 60,2),' mns')
            
    # format resulting dataframe
    output['grid_osm'] = mat5
    output = pd.merge(output, road_nodes[l]['geometry'], left_on = 'grid_osm', right_index = True)
    output['geometry_m'] = gpd.GeoSeries(output['geometry'], crs = 4326).to_crs(3043)
    #output['grid_entry_dist'] = round(gpd.GeoSeries(output['Grid_m_centroid'], crs = 3043
    #                                               ).distance(output['geometry_m']),3)
    output = output.reset_index()
    print('100 % gridentry done', round((time.time() - start_time) / 60,2),' mns')
    RoadComb.append(output)
RoadComb[0]    

Philadelphia
10 / 33 comb. done 6.87  mns
of 6140640 within a Gravity model variant in one of [300, 600, 1000] m threshold: 92734
20 / 33 comb. done 13.83  mns
of 6140640 within a Gravity model variant in one of [300, 600, 1000] m threshold: 36286
30 / 33 comb. done 20.82  mns
of 6140640 within a Gravity model variant in one of [300, 600, 1000] m threshold: 70880
33 / 33 comb. done 22.42  mns
of 2640 within a Gravity model variant in one of [300, 600, 1000] m threshold: 1309
33 / 33
total combinations within distance 0
0.0 % gridentry done 22.42  mns
18.4 % gridentry done 22.81  mns
36.8 % gridentry done 23.21  mns
55.2 % gridentry done 23.59  mns
73.6 % gridentry done 23.98  mns
92.0 % gridentry done 24.35  mns
100 % gridentry done 24.56  mns


,index,raw euclidean,size_infl_eucl2,size_infl_eucl3,size_infl_eucl5,Grid_No,Park_entry_No,Grid_coords_centroid,Grid_m_centroid,Park_No,...,Park_geom,Parkroad_coords_centroid,Parkroad_m_centroid,walk_area_m2,size_infl_sqr2,size_infl_sqr3,size_infl_sqr5,grid_osm,geometry,geometry_m
0,18440,1309.842022,997.374157,1092.223996,1174.558348,18440,0,POINT (-75.15083 39.96417),POINT (-5708413.600 8467498.115),0,...,POINT (-75.14730 39.95683),"MULTIPOLYGON (((-75.14967 39.94550, -75.14974 ...",POINT (-5709440.723 8466685.277),129260.205233,1.313291,1.199243,1.115178,110187995,POINT (-75.15106 39.96424),POINT (-5708411.179 8467529.296)
1,123110,1388.380138,949.790307,1077.923532,1192.768410,18440,3,POINT (-75.15083 39.96417),POINT (-5708413.600 8467498.115),0,...,POINT (-75.14763 39.95625),"MULTIPOLYGON (((-75.14967 39.94550, -75.14974 ...",POINT (-5709546.213 8466695.127),160141.739542,1.461775,1.288014,1.163998,110187995,POINT (-75.15106 39.96424),POINT (-5708411.179 8467529.296)
2,158000,1298.078633,965.896661,1065.910590,1153.328082,18440,4,POINT (-75.15083 39.96417),POINT (-5708413.600 8467498.115),0,...,POINT (-75.14749 39.95686),"MULTIPOLYGON (((-75.14967 39.94550, -75.14974 ...",POINT (-5709444.553 8466709.353),135357.990228,1.343910,1.217812,1.125507,110187995,POINT (-75.15106 39.96424),POINT (-5708411.179 8467529.296)
3,192890,1246.439697,902.086850,1004.745320,1095.221862,18440,5,POINT (-75.15083 39.96417),POINT (-5708413.600 8467498.115),0,...,POINT (-75.14903 39.95686),"MULTIPOLYGON (((-75.14967 39.94550, -75.14974 ...",POINT (-5709506.409 8466898.631),143083.338564,1.381729,1.240553,1.138071,110187995,POINT (-75.15106 39.96424),POINT (-5708411.179 8467529.296)
4,262670,1353.687046,923.303774,1048.904217,1161.579150,18440,7,POINT (-75.15083 39.96417),POINT (-5708413.600 8467498.115),0,...,POINT (-75.14915 39.95619),"MULTIPOLYGON (((-75.14967 39.94550, -75.14974 ...",POINT (-5709617.496 8466879.162),161098.148275,1.466134,1.290573,1.165385,110187995,POINT (-75.15106 39.96424),POINT (-5708411.179 8467529.296)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1358949,781953,662.922514,4080.053542,2226.374220,1371.336408,14373,5478,POINT (-75.16083 39.98917),POINT (-5704829.895 8470016.577),438,...,POINT (-75.16295 39.98556),"POLYGON ((-75.16260 39.98542, -75.16266 39.985...",POINT (-5705488.853 8470088.968),1978.504477,0.162479,0.297759,0.483413,109978485,POINT (-75.16070 39.98849),POINT (-5704931.473 8469964.936)
1358950,782166,538.771429,3315.947537,1809.422361,1114.514684,14586,5478,POINT (-75.16083 39.98833),POINT (-5704962.599 8469973.507),438,...,POINT (-75.16295 39.98556),"POLYGON ((-75.16260 39.98542, -75.16266 39.985...",POINT (-5705488.853 8470088.968),1978.504477,0.162479,0.297759,0.483413,109978485,POINT (-75.16070 39.98849),POINT (-5704931.473 8469964.936)
1358951,781954,713.739521,4392.814240,2397.039224,1476.457609,14374,5478,POINT (-75.16000 39.98917),POINT (-5704796.765 8469914.503),438,...,POINT (-75.16295 39.98556),"POLYGON ((-75.16260 39.98542, -75.16266 39.985...",POINT (-5705488.853 8470088.968),1978.504477,0.162479,0.297759,0.483413,109978478,POINT (-75.16013 39.98842),POINT (-5704920.579 8469891.458)
1358952,782164,468.470453,2883.269898,1573.321944,969.088504,14584,5478,POINT (-75.16250 39.98833),POINT (-5705028.856 8470177.664),438,...,POINT (-75.16295 39.98556),"POLYGON ((-75.16260 39.98542, -75.16266 39.985...",POINT (-5705488.853 8470088.968),1978.504477,0.162479,0.297759,0.483413,109978498,POINT (-75.16227 39.98870),POINT (-5704961.675 8470168.692)


In [14]:
# Number of combinations within range per city
for i in range(len(RoadComb)): print(cities[i], len(RoadComb[i]))

Philadelphia 1358954


In [ ]:
# Block 7 calculate route networks of all grid-parkentry combinations within euclidean threshold distance

warnings.filterwarnings("ignore")
start_time = time.time()

Routes = list()
for j in range(len(cities)):
    Graph = graphs[j]
    CityRoads = RoadComb[j] # iloc to test the iteration speed.
    PR = road_nodes[j]
    
    block = block_network

    Route_parts = pd.DataFrame()
    len2 = int(np.ceil(len(CityRoads)/block))
    1217624
    # Divide in chunks of block for computational load
    for k in range(len2):    
        CityRoad = CityRoads.iloc[k*block:k*block+block]

        parknode = list(CityRoad['Parkroad_osmid'])
        gridnode = list(CityRoad['grid_osm'])
        
        s_mat = list([])
        s_mat1 = list([])
        s_mat2 = list([])
        s_mat3 = list([])
        s_mat4 = list([])
        s_mat5 = list([])
        mat_nn = []
        len1 = len(CityRoad)
        
        print(cities[j].rsplit(',')[0], k+1,'/',len2,'range',k*block,'-',k*block+np.where(k*block+block >= len1,len1,block))
        for i in range(len(CityRoad)):
            try:
                shortest = nx.shortest_path(Graph, gridnode[i], parknode[i], 'travel_dist', method = 'dijkstra')
                s_mat.append(shortest)
                shortest_to = list(shortest[1:len(shortest)])
                shortest_to.append(-1)
                s_mat1.append(shortest_to)
                s_mat2.append(list(np.repeat(i+block*k, len(shortest))))
                s_mat3.append(list(np.arange(0, len(shortest))))
                s_mat4.append('normal way')
                s_mat5.append(1)
            except:
                try:
                    # Check the reverse
                    shortest = nx.shortest_path(Graph, parknode[i], gridnode[i], 'travel_dist', method = 'dijkstra')
                    s_mat.append(shortest)
                    shortest_to = list(shortest[1:len(shortest)])
                    shortest_to.append(-1)
                    s_mat1.append(shortest_to)
                    s_mat2.append(list(np.repeat(i+block*k, len(shortest))))
                    s_mat3.append(list(np.arange(0, len(shortest))))
                    s_mat4.append('reverse way')
                    s_mat5.append(0)
                except:
                    # Otherwise the nearest node is taken, which is iterated X times at max, check assumptions, block #0 
                    # Order in route for nearest node:
                    # 1. gridnode to nearest to the original failed parknode
                    # 2. The reverse of 1.
                    # 3. nearest gridnode to the failed one and route to park
                    # 4. The reverse of 3.

                    len3 = 0
                    alt_route = list([])
                    while len3 < nn_iter and len(alt_route) < 1:

                        len3 = len3 +1
                        # Grid nearest
                        g_geom = PR[PR['osmid_var'] == int(CityRoad.iloc[i:i+1]['grid_osm'])]['geometry']
                        g_nearest = pd.DataFrame((abs(float(g_geom.x) - PR['geometry'].x)**2
                        +abs(float(g_geom.y) - PR['geometry'].y)**2)**(1/2)
                                                ).join(PR['osmid_var']).sort_values(0)

                        g_grid = g_nearest.iloc[len3,1]
                        g_park = CityRoad.iloc[i]['Parkroad_osmid']

                        p_geom = PR[PR['osmid_var'] == int(CityRoad.iloc[i:i+1]['Parkroad_osmid'])]['geometry']
                        p_nearest = pd.DataFrame((abs(float(p_geom.x) - PR['geometry'].x)**2
                        +abs(float(p_geom.y) - PR['geometry'].y)**2)**(1/2)
                                                ).join(PR['osmid_var']).sort_values(0)

                        p_grid = CityRoad.iloc[i]['grid_osm']
                        p_park = p_nearest.iloc[len3,1]

                        try:
                            alt_route.append(nx.shortest_path(Graph, p_grid, p_park, 
                                                              'travel_dist', method = 'dijkstra'))
                            s_mat4.append(str(len3)+'grid > n-park')
                            s_mat5.append(1)
                        except:
                            try:
                                alt_route.append(nx.shortest_path(Graph, p_park, p_grid, 
                                                                  'travel_dist', method = 'dijkstra'))
                                s_mat4.append(str(len3)+'n-park > grid')
                                s_mat5.append(0)
                            except:
                                try:
                                    alt_route.append(nx.shortest_path(Graph, g_grid, g_park, 
                                                                      'travel_dist', method = 'dijkstra'))
                                    s_mat4.append(str(len3)+'n-grid > park')
                                    s_mat5.append(1)
                                except:
                                    try:
                                        alt_route.append(nx.shortest_path(Graph, g_grid, g_park, 
                                                                          'travel_dist', method = 'dijkstra'))
                                        s_mat4.append(str(len3)+'park > n-grid')
                                        s_mat5.append(0)
                                    except:
                                        if len3 == nearest_node_iterations:
                                            #print(i+block*k,i+block*k,
                                            #      'No route between grid and park-entry and their both 10 alternatives')
                                            pass
                                        pass
                    #print(len(alt_route))
                    if len(alt_route) == 0: 
                        alt = alt_route 
                    else: 
                        alt = alt_route[0]
                    len4 = len(alt)
                    #print(len4)
                    #mat_nn = []
                    if len4 > 0:
                        #print('for index',i+block*k,'nearest node found between', 
                        #                       alt[0],'and',alt[-1])
                        mat_nn.append(i+block*k)
                        s_mat.append(alt)
                        shortest_to = list(alt[1:len(alt)])
                        shortest_to.append(-1)
                        s_mat1.append(shortest_to)
                        s_mat2.append(list(np.repeat(i+block*k,len4)))
                        s_mat3.append(list(np.arange(0, len4)))
                    else:
                        s_mat.append(-1)
                        s_mat1.append(-1)
                        s_mat2.append(i+block*k)
                        s_mat3.append(-1)
                        s_mat4.append('no way')
                        s_mat5.append(2)
                        print(i+block*k,'No route between grid and park-entry and their both',nn_iter,'alternatives')

            if i % 10000 == 0: print(round((i+block*k)/len(CityRoads)*100,2),'% done',
                                     round((time.time() - start_time) / 60,2),'mns')
        print(len(mat_nn),'nearest nodes found')

        print(round((i+block*k)/len(CityRoads)*100,2),'% pathfinding done', round((time.time() - start_time) / 60,2),'mns')
                          
        # Unpack lists
        s_mat_u = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, s_mat) for i in b]
        s_mat_u1 = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, s_mat1) for i in b]
        s_mat_u2 = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, s_mat2) for i in b]
        s_mat_u3 = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, s_mat3) for i in b]

        # Format df
        routes = pd.DataFrame([s_mat_u,s_mat_u1,s_mat_u2,s_mat_u3]).transpose()
        routes.columns = ['from','to','route','step']
        mat_key = list([])
        for n in range(len(routes)):
            mat_key.append(str(int(s_mat_u[n])) + '-' + str(int(s_mat_u1[n])))
        routes['key'] = mat_key
        routes = routes.set_index('key')

        # Add route information
        routes = routes.join(road_conn[j], how = 'left')
        routes = gpd.GeoDataFrame(routes, geometry = 'geometry', crs = 4326)
        print('formatting done', round((time.time() - start_time) / 60,2), 'mns')
        routes = routes.sort_values(by = ['route','step'])

        # dissolve route
        routes2 = routes[['route','geometry']].dissolve('route')
        
        # get used grid- and parkosm. Differs at NN-route.
        route_reset = routes.reset_index()
        origin = route_reset['from'].iloc[list(route_reset.groupby('route')['step'].idxmin()),]
        origin = origin.reset_index().iloc[:,-1]
        dest = route_reset['from'].iloc[list(route_reset.groupby('route')['step'].idxmax()),]
        dest = dest.reset_index().iloc[:,-1]
        
        # grid > park = 1, park > grid = 0, no way = 2, detailed way in way_calc.
        routes2['G1-P0'] = s_mat5
        routes2['real_grid'] = np.where(routes2['G1-P0'] == 1, origin, dest)
        routes2['real_park'] = np.where(routes2['G1-P0'] == 1, dest, origin)
        routes2['way_calc.'] = s_mat4
        
        # get route cost, steps, additional information.
        routes2['route_cost'] = routes.groupby('route')['length'].sum()
        routes2['steps'] = routes.groupby('route')['step'].max()
        routes2['index'] = CityRoad.index
        routes2 = routes2.set_index(['index'])
        routes2.index = routes2.index.astype(int)
        routes2 = pd.merge(routes2, CityRoad[['Grid_No','grid_osm','Park_No','Park_entry_No','Parkroad_osmid',
                                              'Grid_m_centroid','walk_area_m2','size_infl_sqr2','size_infl_sqr3',
                                              'size_infl_sqr5','raw euclidean']],
                                                left_index = True, right_index = True)
        routes2 = pd.merge(routes2, road_nodes[j]['geometry_m'], how = 'left', left_on = 'real_grid', right_index = True)
        
        # calculate distance of used road-entry for grid-centroid.
        routes2['real_G-entry'] = round(gpd.GeoSeries(routes2['Grid_m_centroid'], crs = 3043
                                                       ).distance(routes2['geometry_m']),3)
        
        # Calculcate total route cost for the four gravity variants
        routes2['raw_Tcost'] = routes2['route_cost'] + routes2['real_G-entry']
        routes2['grav2_Tcost'] = (routes2['route_cost'] + routes2['real_G-entry']) / routes2['size_infl_sqr2']
        routes2['grav3_Tcost'] = (routes2['route_cost'] + routes2['real_G-entry']) / routes2['size_infl_sqr3']
        routes2['grav5_Tcost'] = (routes2['route_cost'] + routes2['real_G-entry']) / routes2['size_infl_sqr5']

        routes2['gridpark_no'] = routes2['Grid_No'].astype(str) +'-'+ routes2['Park_No'].astype(str)
        print('dissolving done', round((time.time() - start_time) / 60,2), 'mns')
        Route_parts = pd.concat([Route_parts, routes2])
    Routes.append(Route_parts)
Routes[0]
# 60m Philadelphia

In [18]:
Routes[0]

,geometry,G1-P0,real_grid,real_park,way_calc.,route_cost,steps,Grid_No,grid_osm,Park_No,...,size_infl_sqr3,size_infl_sqr5,raw euclidean,geometry_m,real_G-entry,raw_Tcost,grav2_Tcost,grav3_Tcost,grav5_Tcost,gridpark_no
index,,,,,,,,,,,,,,,,,,,,,
0,"MULTILINESTRING ((-75.15106 39.96424, -75.1512...",1,110187995,109729372,normal way,1245.729,26,18440,110187995,0,...,1.199243,1.115178,1309.842022,POINT (-5708411.179 8467529.296),31.275,1277.004,972.369771,1064.841705,1145.111917,18440-0
1,"MULTILINESTRING ((-75.15106 39.96424, -75.1512...",1,110187995,109729661,normal way,1460.015,29,18440,110187995,0,...,1.288014,1.163998,1388.380138,POINT (-5708411.179 8467529.296),31.275,1491.290,1020.190904,1157.821651,1281.179091,18440-0
2,"MULTILINESTRING ((-75.15106 39.96424, -75.1512...",1,110187995,109729673,normal way,1229.607,25,18440,110187995,0,...,1.217812,1.125507,1298.078633,POINT (-5708411.179 8467529.296),31.275,1260.882,938.218751,1035.366766,1120.279297,18440-0
3,"MULTILINESTRING ((-75.15106 39.96424, -75.1512...",1,110187995,109760667,normal way,1122.249,20,18440,110187995,0,...,1.240553,1.138071,1246.439697,POINT (-5708411.179 8467529.296),31.275,1153.524,834.840894,929.846701,1013.578680,18440-0
4,"MULTILINESTRING ((-75.15106 39.96424, -75.1512...",1,110187995,109795250,normal way,1197.124,22,18440,110187995,0,...,1.290573,1.165385,1353.687046,POINT (-5708411.179 8467529.296),31.275,1228.399,837.849070,951.824793,1054.071302,18440-0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1358949,"MULTILINESTRING ((-75.16070 39.98849, -75.1610...",1,109978485,110257674,normal way,489.453,6,14373,109978485,438,...,0.297759,0.483413,662.922514,POINT (-5704931.473 8469964.936),113.952,603.405,3713.744298,2026.489231,1248.217140,14373-438
1358950,"MULTILINESTRING ((-75.16070 39.98849, -75.1610...",1,109978485,110257674,normal way,489.453,6,14586,109978485,438,...,0.297759,0.483413,538.771429,POINT (-5704931.473 8469964.936),32.284,521.737,3211.106651,1752.213541,1079.276881,14586-438
1358951,"MULTILINESTRING ((-75.16013 39.98842, -75.1599...",1,109978478,110257674,normal way,615.192,10,14374,109978478,438,...,0.297759,0.483413,713.739521,POINT (-5704920.579 8469891.458),125.941,741.133,4561.411412,2489.038114,1533.124376,14374-438


In [15]:
# Block 8 determine best parkentry points from each grid, then calculate grid scores
# and finally aggregate city access in categories (high, medium, low and no access)

start_time = time.time()
popg_acc = pd.DataFrame()
adj_popg_acc = pd.DataFrame()
grid_scores = list([])
gridpark = list([])
for n in range(len(cities)):    
    print(cities[n])
    
    # For the four distance decay variants regarding park size.
    l1 = list(['raw','grav2','grav3','grav5'])
    m1 = list(['entrance','gravity**(1/2)','gravity**(1/3)','gravity**(1/5)'])
    grid_score = list([])
    gridparks = list([])
    gridpark.append(gridparks)
    popgrid_access = pd.DataFrame()
    adj_popgrid_access = pd.DataFrame()
    for i in range(len(l1)):
        # Get the lowest indices grouped by a key consisting of grid no and park no (best entry point from a grid to a park)
        Rclean = Routes[n][Routes[n]['way_calc.'] != 'no way'].reset_index()
        str1 = 'gridpark_' + l1[i]
        locals()[str1] = Rclean.iloc[Rclean.groupby('gridpark_no')[(str(l1[i]) +'_Tcost')].idxmin()]
        l2 = list()
        
        # Get the total cost column
        for j in enumerate(l1): 
            if j[0] != i: l2.append(j[1] + '_Tcost')
        locals()[str1] = locals()[str1].loc[:, ~locals()[str1].columns.isin(l2)]
        
        # Get grid information
        locals()[str1] = pd.merge(locals()[str1], grids[n][['population','geometry']],
                                left_on = 'Grid_No', right_index = True, how = 'outer')
        locals()[str1] = locals()[str1].reset_index()
        
        # formatting
        locals()[str1]['Park_No'] = locals()[str1]['Park_No'].fillna(-1)
        locals()[str1]['Park_No'] = locals()[str1]['Park_No'].astype(int)
        locals()[str1]['Park_entry_No'] = locals()[str1]['Park_entry_No'].fillna(-1)
        locals()[str1]['Park_entry_No'] = locals()[str1]['Park_entry_No'].astype(int)
        
        grdsc = pd.DataFrame()
        gridsc = pd.DataFrame()
        print(m1[i], round((time.time() - start_time) / 60,2), 'mns')
        
        # For each threshold given, calculate a score
        for k in range(len(thresholds)):
            t = thresholds[k]
            str2 = str(t)
            score = 'tr_'+ str2
            adj_score = 'adj_tr_' + str2
            
            #Only get routes within the threshold given (it loops over every threshold) and calculate the scores
            thold = locals()[str1][locals()[str1][l1[i] + '_Tcost'] <= t]
            thold[score] = t - thold[l1[i] + '_Tcost']
            thold['pop' + score] = thold[score] * thold['population']
            thold[adj_score] = thold[score] * ((math.pi*600**2) / (math.pi*t**2)) / (600/t)
            thold['walk_area_ha' + str2] = locals()[str1]['walk_area_m2'] /10000
            thold['walkha_person' + str2] = thold['population'] / thold['walk_area_ha' + str2]
            
            # Join the gridpark information from before.
            locals()[str1] = locals()[str1].join(thold[[score,'pop' + score,adj_score,'walk_area_ha' + str2, 'walkha_person' + str2]])
            # get the grid_scores
            gs = pd.DataFrame()
            gs[[score,'pop_' + score,adj_score,'walkha_' + str2]] = locals()[str1].groupby(
                    'Grid_No')[score,'pop' + score, adj_score, 'walk_area_ha' + str2].sum()
            
            gs['walkha_person_' + score] = locals()[str1].groupby('Grid_No')['walkha_person' + str2].mean()

            trstr = locals()[str1][locals()[str1][score] > 0]
            gs[score + '_parks'] = trstr.groupby('Grid_No')['gridpark_no'].count()
            
            # Add the routes as a dissolved line_geom
            gs[score + '_routes'] = gpd.GeoDataFrame(trstr[['Grid_No','geometry_x']],
                                                          geometry = 'geometry_x', crs = 4326).dissolve('Grid_No')

            # Add parks which grids have access to with its closest access point
            gs[score+'Park:entry'] = trstr[trstr['Park_No'] >=0].groupby('Grid_No')['Park_No'].apply(list).astype(str
            ) + ':' + trstr[trstr['Park_entry_No'] >=0].groupby('Grid_No')['Park_entry_No'].apply(list).astype(str)
            
            # determine the thresholds category-score. 
            # High >= threshold (perfect score to one park), medium is above half perfect, 
            # low is below this and no is no access to a park for a certain grid within the threshold given
            gs[score+'_access'] = np.select([gs[score] >= t, (gs[score] < t) & (
            gs[score]>= t/2), (gs[score] < t/2) & (gs[score]> 0), gs[score] <= 0],
                  ['1 high','2 medium','3 low','4 no'])
                        
            gs[score+'_adj_access'] = np.select([gs[adj_score] >= t, (gs[adj_score] < t) & (
            gs[adj_score]>= t/2), (gs[adj_score] < t/2) & (gs[adj_score]> 0), gs[adj_score] <= 0],
                  ['1 high','2 medium','3 low','4 no'])
            
            gs = gs.join(grids[n]['population'], how = 'outer')
            grdsc = pd.concat([grdsc, gs], axis = 1)
            
            gs = gpd.GeoDataFrame(gs, geometry = score + '_routes', crs = 4326)
            
            if not os.path.exists('D:Dumps/Scores output WP2-OSM/'+str(grid_block_size)+'m grids/Grid_lines/'):
                os.makedirs('D:Dumps/Scores output WP2-OSM/'+str(grid_block_size)+'m grids/Grid_lines/')
                
            gs.to_file('D:Dumps/Scores output WP2-OSM/'+str(grid_block_size)+'m grids/Grid_lines/gridscore_'+ l1[i] + '_' + str2 + '_' + cities[n] + '.shp')
            
            gsc = gs.loc[:,~gs.columns.isin([score + '_routes'])]
            gridsc = pd.concat([gridsc, gsc])

            # Group according to the categories just created and sum the populations living in those grids
            popgacc = pd.DataFrame()
            popgacc[m1[i]+'_'+str(t)] = gs.groupby(score+'_access')['population'].sum()
            popgrid_access = pd.concat([popgrid_access, popgacc],axis=1)   
            
            adj_popgacc = pd.DataFrame()
            adj_popgacc[m1[i]+'_'+str(t)] = gs.groupby(score+'_adj_access')['population'].sum()
            adj_popgrid_access = pd.concat([adj_popgrid_access, adj_popgacc],axis=1)   
            print('grid ',t)
        
        grid_score.append(grdsc)
        
        gridsc = gridsc.join(grids[n]['geometry'])
        gridsc = gpd.GeoDataFrame(gridsc, geometry = 'geometry', crs = 4326)
        
        if not os.path.exists('D:Dumps/Scores output WP2-OSM/'+str(grid_block_size)+'m grids/Grid_geoms/'):
            os.makedirs('D:Dumps/Scores output WP2-OSM/'+str(grid_block_size)+'m grids/Grid_geoms/')
            
        gridsc.to_file('D:Dumps/Scores output WP2-OSM/'+str(grid_block_size)+'m grids/Grid_geoms/gridscore_'+ l1[i] + '_' + cities[n] + '.shp')
        
        # Detailed scores to files number of cities * ways to measure = number of files.
        # Different threshold-scores are in the same dataframe
        gridsc = gridsc.loc[:, gridsc.columns!='geometry']
        
        if not os.path.exists('D:Dumps/Scores output WP2-OSM/'+str(grid_block_size)+'m grids/Grid_csv/'):
            os.makedirs('D:Dumps/Scores output WP2-OSM/'+str(grid_block_size)+'m grids/Grid_csv/')
        
        gridsc.to_csv('D:/Dumps/Scores output WP2-OSM/'+str(grid_block_size)+'m grids/Grid_csv/gridscore_'+ l1[i] + '_' + cities[n] + '.csv')
        gridparks.append(locals()[str1])
        
    grid_scores.append(grid_score)

    # For each city, divide the population access by group by the total to get its share.
    popgrid_access = popgrid_access / popgrid_access.sum()
    popgrid_access = pd.DataFrame(popgrid_access.unstack())
    popg_acc = pd.concat([popg_acc, popgrid_access], axis = 1)
    
    adj_popgrid_access = adj_popgrid_access / adj_popgrid_access.sum()
    adj_popgrid_access = pd.DataFrame(adj_popgrid_access.unstack())
    adj_popg_acc = pd.concat([adj_popg_acc, adj_popgrid_access], axis = 1)
    
    print(cities[n],'done', round((time.time() - start_time) / 60,2), 'mns')
popg_acc.columns = cities
adj_popg_acc.columns = cities

popg_acc.to_csv('D:/Dumps/Scores output WP2-OSM/'+str(grid_block_size)+'m grids/popgrid_access.csv')
adj_popg_acc.to_csv('D:/Dumps/Scores output WP2-OSM/'+str(grid_block_size)+'m grids/radius-euclidean adjusted popgrid access.csv')

popg_acc    


Philadelphia
entrance 0.13 mns
grid  300
grid  600
grid  1000
gravity**(1/2) 1.92 mns
grid  300
grid  600
grid  1000
gravity**(1/3) 3.62 mns
grid  300
grid  600
grid  1000
gravity**(1/5) 5.32 mns
grid  300
grid  600
grid  1000
Philadelphia done 6.95 mns


Philadelphia
entrance_300        1 high        0.006859
                    2 medium      0.105432
                    3 low         0.184383
                    4 no          0.703326
entrance_600        1 high        0.098546
                    2 medium      0.268793
                    3 low         0.317859
                    4 no          0.314802
entrance_1000       1 high        0.410893
                    2 medium      0.288735
                    3 low         0.191523
                    4 no          0.108850
gravity**(1/2)_300  1 high        0.002107
                    2 medium      0.103606
                    3 low         0.163829
                    4 no          0.730458
gravity**(1/2)_600  1 high        0.033122
                    2 medium      0.266787
                    3 low         0.327739
                    4 no          0.372351
gravity**(1/2)_1000 1 high        0.221262
                    2 medium      0.413076
                    3 low         0.249031
                    4 no          0.116632
gravity**(1/3)_300  1 high        0.002113
                    2 medium      0.097520
                    3 low         0.156511
                    4 no          0.743855
gravity**(1/3)_600  1 high        0.033718
                    2 medium      0.250972
                    3 low         0.325913
                    4 no          0.389398
gravity**(1/3)_1000 1 high        0.225735
                    2 medium      0.388440
                    3 low         0.273224
                    4 no          0.112601
gravity**(1/5)_300  1 high        0.002448
                    2 medium      0.095254
                    3 low         0.157702
                    4 no          0.744595
gravity**(1/5)_600  1 high        0.043829
                    2 medium      0.247516
                    3 low         0.331700
                    4 no          0.376955
gravity**(1/5)_1000 1 high        0.265903
                    2 medium      0.365753
                    3 low         0.257873
                    4 no          0.110471

In [16]:
# Block 9 calculte park scores from previously determined best grid-park routes.

start_time = time.time()
cityparks = list([])
for i in range(len(cities)):
    
    # For the four distance decay variants regarding park size.
    l1 = list(['raw','grav2','grav3','grav5'])
    m1 = list(['entrance','gravity**(1/2)','gravity**(1/3)','gravity**(1/5)'])
    parks = list([])
    for j in range(len(l1)):
        parksc = pd.DataFrame()
        prksc = pd.DataFrame()
        for k in range(len(thresholds)):
            score = 'tr_' + str(thresholds[k])
            str2 = str(thresholds[k])
            str1 = gridpark[i][j][gridpark[i][j][score] > 0]

            # Get the park scores
            prk = pd.DataFrame()
            prk[[score,'pop_' + score,'popreach'+score]] = str1.groupby('Park_No')[score,'pop' + score,'population'].sum()
            
            prk['walkha_person_' + score] = str1.groupby('Park_No')['walkha_person' + str2].mean()
            prk[score + '_parks'] = str1.groupby('Park_No')['gridpark_no'].count()
            
            # Add the routes as a dissolved line_geom
            prk[score+'route'] = gpd.GeoDataFrame(str1[['Park_No','geometry_x']], 
                             geometry = 'geometry_x', crs = 4326).dissolve('Park_No')
            
            # Add parks which grids have access to with its closest access point
            prk[score+'Grid:Pentry'] = str1[str1['Grid_No'] >=0].groupby('Park_No')['Grid_No'].apply(list).astype(str
            ) + ':' + str1[str1['Park_entry_No'] >=0].groupby('Park_No')['Park_entry_No'].apply(list).astype(str)
            
            # Get all parks, even with no score.
            prk = prk.join(parks_in_range[i].iloc[:,0], how = 'outer')
            prk = prk.loc[:,~prk.columns.isin(['components'])]
            print('park', thresholds[k])
            
            # Get the park score categories (same as grid score)
            prk[score+'_access'] = np.select([prk[score] >= t, (prk[score] < t) & (
                prk[score]>= t/2), (prk[score] < t/2) & (prk[score]> 0), prk[score] <= 0 | prk[score].isna()],
                ['1 high','2 medium','3 low','4 no'])

            parksc = pd.concat([parksc, prk], axis = 1)
                
            prk = gpd.GeoDataFrame(prk, geometry = score+'route', crs = 4326)
        
            if not os.path.exists('D:Dumps/Scores output WP2-OSM/'+str(grid_block_size)+'m grids/Park_lines/'):
                os.makedirs('D:Dumps/Scores output WP2-OSM/'+str(grid_block_size)+'m grids/Park_lines/')
            
            prk.to_file('D:Dumps/Scores output WP2-OSM/'+str(grid_block_size)+'m grids/Park_lines/parkscore_'+ l1[j] + '_' + str2 + '_' + cities[i] + '.shp')
            
            psc = prk.loc[:,~prk.columns.isin([score + 'route'])]
            prksc = pd.concat([prksc, psc])
            
        parks.append(parksc)
        
        prksc = prksc.join(parks_in_range[i]['geometry'])
        prksc = gpd.GeoDataFrame(prksc, geometry = 'geometry', crs = 4326)
        
        if not os.path.exists('D:Dumps/Scores output WP2-OSM/'+str(grid_block_size)+'m grids/Park_geoms/'):
            os.makedirs('D:Dumps/Scores output WP2-OSM/'+str(grid_block_size)+'m grids/Park_geoms/')
            
        prksc.to_file('D:Dumps/Scores output WP2-OSM/'+str(grid_block_size)+'m grids/Park_geoms/parkscore_'+ l1[j] + '_' + cities[i] + '.shp')
        
        # Detailed scores to files number of cities * ways to measure = number of files.
        # Different threshold-scores are in the same dataframe
        prksc = prksc.loc[:, prksc.columns!='geometry']
        
        if not os.path.exists('D:Dumps/Scores output WP2-OSM/'+str(grid_block_size)+'m grids/Park_csv/'):
            os.makedirs('D:Dumps/Scores output WP2-OSM/'+str(grid_block_size)+'m grids/Park_csv/')
            
        prksc.to_csv('D:/Dumps/Scores output WP2-OSM/'+str(grid_block_size)+'m grids/Park_csv/parkscore_'+ l1[j] + '_' + cities[i]+ '.csv')

        print(m1[j], round((time.time() - start_time) / 60,2), 'mns')
    cityparks.append(parks)
    print(cities[i],'done', round((time.time() - start_time) / 60,2), 'mns')
pd.DataFrame(cityparks[0][0])

park 300
park 600
park 1000
entrance 0.56 mns
park 300
park 600
park 1000
gravity**(1/2) 2.14 mns
park 300
park 600
park 1000
gravity**(1/3) 2.85 mns
park 300
park 600
park 1000
gravity**(1/5) 3.39 mns
Philadelphia done 3.39 mns


,tr_300,pop_tr_300,popreachtr_300,walkha_person_tr_300,tr_300_parks,tr_300route,tr_300Grid:Pentry,tr_300_access,tr_600,pop_tr_600,...,tr_600Grid:Pentry,tr_600_access,tr_1000,pop_tr_1000,popreachtr_1000,walkha_person_tr_1000,tr_1000_parks,tr_1000route,tr_1000Grid:Pentry,tr_1000_access
0,22819.670,1687100.947,10517.0,11.823637,141.0,"(LINESTRING (-75.1522432 39.957596, -75.152167...","[19289, 19290, 19291, 19293, 19377, 19378, 193...",1 high,84113.441,6224254.580,...,"[18813, 18913, 19012, 19013, 19014, 19015, 191...",1 high,240734.599,1.690499e+07,33023.0,10.869332,505.0,"(LINESTRING (-75.149165 39.964968, -75.148399 ...","[18328, 18329, 18330, 18443, 18447, 18605, 186...",1 high
1,5632.736,312757.069,2821.0,11.271763,44.0,"(LINESTRING (-75.1537 39.935895, -75.153774 39...","[21902, 21903, 21904, 21906, 21907, 21908, 219...",1 high,29023.436,1870862.350,...,"[21479, 21624, 21627, 21628, 21721, 21722, 217...",1 high,104638.671,7.724820e+06,18208.0,13.506889,237.0,"(LINESTRING (-75.1560085 39.9400095, -75.15609...","[21385, 21388, 21389, 21390, 21391, 21393, 214...",1 high
2,3191.788,260601.998,2444.0,29.370938,27.0,"(LINESTRING (-75.151595 39.9307953, -75.151674...","[22426, 22582, 22583, 22584, 22668, 22670, 226...",1 high,19750.807,1860557.022,...,"[22256, 22342, 22343, 22344, 22345, 22422, 224...",1 high,76538.927,7.187005e+06,17357.0,29.641586,190.0,"(LINESTRING (-75.1516317 39.9345242, -75.15088...","[22081, 22082, 22162, 22164, 22167, 22168, 221...",1 high
3,3317.927,148928.776,1239.0,25.961469,25.0,"(LINESTRING (-75.1413467 39.9681793, -75.14140...","[17879, 18000, 18001, 18114, 18115, 18116, 181...",1 high,17897.466,840928.008,...,"[17637, 17638, 17639, 17760, 17761, 17762, 177...",1 high,73433.202,3.059974e+06,7273.0,20.705882,184.0,"(LINESTRING (-75.142581 39.972388, -75.1426222...","[17193, 17194, 17389, 17390, 17391, 17392, 173...",1 high
4,1656.182,221043.560,2126.0,70.371693,16.0,"(LINESTRING (-75.1691688 39.9641609, -75.16917...","[18423, 18425, 18589, 18590, 18591, 18593, 186...",1 high,11227.045,1338438.529,...,"[18089, 18203, 18309, 18311, 18312, 18313, 183...",1 high,47231.274,4.824604e+06,12248.0,51.075918,127.0,"(LINESTRING (-75.1707792 39.9672738, -75.17136...","[17967, 17968, 17969, 17970, 17971, 17972, 179...",1 high
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
444,1284.784,67094.468,776.0,586.388830,13.0,"(LINESTRING (-75.1675625 40.0072758, -75.16706...","[11520, 11521, 11523, 11524, 11630, 11631, 116...",1 high,10717.381,568525.088,...,"[11094, 11206, 11207, 11314, 11315, 11316, 113...",1 high,46650.651,2.283359e+06,5343.0,464.487521,113.0,"(LINESTRING (-75.167653 40.010918, -75.1674388...","[10979, 10980, 10982, 10983, 10984, 11091, 110...",1 high
445,6273.465,399684.016,3988.0,8.319988,52.0,"(LINESTRING (-75.2108485 39.9516979, -75.21085...","[20046, 20047, 20048, 20154, 20155, 20156, 201...",1 high,35253.597,2627615.175,...,"[19756, 19845, 19846, 19847, 19945, 19946, 199...",1 high,118269.096,9.264830e+06,21734.0,8.671038,271.0,"(LINESTRING (-75.2116212 39.9559681, -75.21168...","[19516, 19518, 19519, 19520, 19522, 19661, 196...",1 high
446,916.244,64030.424,648.0,67.062129,10.0,"(LINESTRING (-75.1625 39.9399604, -75.1625715 ...","[21377, 21464, 21465, 21466, 21612, 21614, 216...",2 medium,10547.711,752222.903,...,"[21122, 21213, 21295, 21298, 21299, 21300, 213...",1 high,53653.556,3.891993e+06,11122.0,75.725410,152.0,"(LINESTRING (-75.164285 39.9439532, -75.164379...","[20917, 20919, 20920, 20921, 20922, 21017, 210...",1 high
447,2308.891,95281.308,1000.0,6.831214,18.0,"(LINESTRING (-75.100552 39.986001, -75.100308 ...","[15098, 15099, 15100, 15236, 15237, 15238, 152...",1 high,13887.659,830079.383,...,"[14649, 14804, 14805, 14806, 14809, 14811, 149...",1 high,52105.559,3.318262e+06,8349.0,8.214623,125.0,"(LINESTRING (-75.1016977 39.9898509, -75.10152...","[14291, 14292, 14433, 14434, 14435, 14436, 144...",1 high


In [ ]:
# Block 10 get the preferenced parks for each grid (lowest score) for all distance decay variants.
preference = list([])
for n in enumerate(cities): 
    print(n[1])
    l1 = list(['raw','grav2','grav3','grav5'])
    m1 = list(['entrance','gravity**(1/2)','gravity**(1/3)','gravity**(1/5)'])
    prefer = list([])
    for j in enumerate(l1):
        pref = list([])
        print(m1[j[0]])
        for k in thresholds:
            score = 'tr_'+ str(k)
            g = gridpark[n[0]][j[0]].iloc[gridpark[n[0]][j[0]].groupby('Grid_No')[score].idxmax().dropna().astype(int)]
            g = g[g[score] > 0]
            g.join(grids[n[0]]['dissolve_key'], how = 'outer')
            g_csv = g.loc[:, (g.columns!='geometry_x') & (g.columns!='geometry_y')]
            
            if not os.path.exists('D:Dumps/Scores output WP2-OSM/'+str(grid_block_size)+'m grids/Grid_pref_parks_csv/'):
                os.makedirs('D:Dumps/Scores output WP2-OSM/'+str(grid_block_size)+'m grids/Grid_pref_parks_csv/')
                
            g_csv.to_csv('D:/Dumps/Scores output WP2-OSM/'+str(grid_block_size)
                         +'m grids/Grid_pref_parks_csv/park-pref_' + j[1] +'-'+ str(k) + '-' + n[1] +'.csv')
            
            g_lines = gpd.GeoDataFrame(g.loc[:, ~g.columns.isin(['geometry_y'])], geometry = 'geometry_x', crs = 4326)
            
            if not os.path.exists('D:Dumps/Scores output WP2-OSM/'+str(grid_block_size)+'m grids/Grid_pref_parks_lines/'):
                os.makedirs('D:Dumps/Scores output WP2-OSM/'+str(grid_block_size)+'m grids/Grid_pref_parks_lines/')
                
            g_lines.to_file('D:Dumps/Scores output WP2-OSM/'+str(grid_block_size)
                            +'m grids/Grid_pref_parks_lines/park-pref_' +'-'+ j[1] + str(k) + '-' + n[1] +'.shp')
            g_geoms = gpd.GeoDataFrame(g.loc[:, ~g.columns.isin(['geometry_x'])], geometry = 'geometry_y', crs = 4326)
            
            if not os.path.exists('D:Dumps/Scores output WP2-OSM/'+str(grid_block_size)+'m grids/Grid_pref_parks_geoms/'):
                os.makedirs('D:Dumps/Scores output WP2-OSM/'+str(grid_block_size)+'m grids/Grid_pref_parks_geoms/')
                
            g_geoms.to_file('D:Dumps/Scores output WP2-OSM/'+str(grid_block_size)
                            +'m grids/Grid_pref_parks_geoms/park-pref_' +'-'+ j[1] + str(k) + '-' + n[1] +'.shp')
            pref.append(g)
            print('park_prefer',k)
        prefer.append(pref)
        len(pref)
    preference.append(prefer)
print('all done')
preference[0][0][0]

In [ ]:
print(round((time.time() - start) / 60,2),'mns')